Ссылка на данный блокнот: https://colab.research.google.com/drive/1Z5fVuajC1rLZQe_w2rJ2LFEzh1SV6KXq?usp=drive_link

Устанавка необходимых модулей, как и в файле "Generation_files.ipynb" для работы Spark.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

import random
import uuid
import string
import hashlib
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import Window
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Объявление схемы данных для дальнейшего считывания JSON файлов и составления Data Frame.

In [2]:
# схема данных
schema = T.StructType([T.StructField('INN', T.StringType(), True),
                       T.StructField('raw_cookie', T.ArrayType(T.MapType(T.StringType(),T.StringType()))),
                       T.StructField('event_type', T.StringType(), True),
                       T.StructField('event_action', T.StringType(), True),
                       T.StructField('data_value', T.StringType(), True),
                       T.StructField('geocountry', T.StringType(), True),
                       T.StructField('city', T.StringType(), True),
                       T.StructField('user_os', T.StringType(), True),
                       T.StructField('systemlanguage', T.StringType(), True),
                       T.StructField('geoaltitude', T.StringType(), True),
                       T.StructField('meta_platform', T.StringType(), True),
                       T.StructField('screensize', T.StringType(), True),
                       T.StructField('timestampcolumn', T.DateType(), True)
                      ])

функции считывания файлов и поиска значений в словарях
UDF функция предназначенная для перевода в md5

In [9]:
path_to_files = '/content/gdrive/MyDrive/data/json/'
filenames = sorted(os.listdir(path_to_files), reverse=True)

def read_file_JSON(count: int=len(filenames), schema: T.StructType=schema):
  blank_DF = spark.createDataFrame([], schema=schema)
  for i in filenames[:count]:
    table = spark.read.format("json")\
                 .load(f"{path_to_files}{i}", schema=schema)
    blank_DF = (blank_DF.union(table)
                      #  .withColumn('rank',
                      #              F.rank().over(Window.partitionBy('INN')
                      #              .orderBy(F.desc('timestampcolumn'))))
                      #  .filter('rank = 1')
                      #  .drop('rank')
                       )
  return blank_DF

def search_values_from_cookie(key):
  return f'filter(raw_cookie, x -> x.key="{key}")'

transformation_values_in_md5 = F.udf(lambda x: hashlib.md5(bytes(str(x), encoding='utf-8')).hexdigest())
transformation_values_in_sha256 = F.udf(lambda x: hashlib.sha256(bytes(str(x), encoding='utf-8')).hexdigest())

@F.udf
def match_code(x):
    return [None,'IDFA','GAID'][1 if x == 'IOS' else 2 if x == 'Android' else 0]

# Создание витрины "A".

In [10]:
data_mart_A = read_file_JSON().drop('INN')
data_mart_A.show(5)

+--------------------+----------+---------------+--------------------+------------------+-------------+------------+--------------+--------------------+-------------+----------+---------------+
|          raw_cookie|event_type|   event_action|          data_value|        geocountry|         city|     user_os|systemlanguage|         geoaltitude|meta_platform|screensize|timestampcolumn|
+--------------------+----------+---------------+--------------------+------------------+-------------+------------+--------------+--------------------+-------------+----------+---------------+
|[{value -> SA1.01...|  REGISTER|login-check-otp|                null|Dominican Republic|Santo Domingo|BlackBerryOS|            RU|     19,-70.66666666|       MOBAIL| 1920x1080|     2024-02-08|
|[{value -> SA1.13...|  REGISTER|       pageview|                null|           Bahrain|       Manama|     Windows|            RU|            26,50.55|          WEB| 1920x1080|     2024-02-08|
|[{value -> SA1.5d...|    SUBM

# Создание витрины "В".

Согласно задания удаляем дубликаты пользователей, которые заходили с одних и тех же устройств. Для создания колонки ID, в данном случае использовалась оконная функция.


In [ ]:
# Создать цикл счтывания DF по датам. удалять дубликать на объединении дней, удалять cookie по INN

data_mart_B = read_file_JSON().select('INN')
data_mart_B = data_mart_B.select(F.row_number()
                                  .over(Window.orderBy(data_mart_B.INN.desc()))
                                  .alias('ID'), 'INN')
data_mart_B.show(5)

+---+------------+
| ID|         INN|
+---+------------+
|  1|999766762140|
|  2|999747435335|
|  3|999079527236|
|  4|998831247489|
|  5|998700646665|
+---+------------+
only showing top 5 rows



# Создание витрины "С"

Для данной и конечной витрины создана функция "search_dict", позволяющая в массиве словарей, найти необходимый словарь по значению ключа. Колонка "ID_С" сформирована с помощью втроеной монотонно увеличивающейся функции "monotonically_increasing_id" в библиотеку "pyspark.sql.functions"

In [ ]:
data_mart_C = read_file_JSON()
data_mart_C = data_mart_C.select((
    F.monotonically_increasing_id() + 1).alias('ID'),
    F.expr(search_values_from_cookie("_sa_cookie_a"))[0]['value']
                                  .alias("sa_cookie_a"))
data_mart_C.show(5, truncate=False)

+---+---------------------------------------------------+
|ID |sa_cookie_a                                        |
+---+---------------------------------------------------+
|1  |SA1.16392d56-84cb-41b3-9321-88fcae7d4a84.6337624765|
|2  |SA1.01e4fc19-fefc-4ad4-9e44-c44ad96d2d02.2612299972|
|3  |SA1.522559b6-45d8-43a6-9302-114da34d80b7.8123392298|
|4  |SA1.d5dfcfec-9a8d-4db8-9124-5de3def4d4cc.2248636980|
|5  |SA1.3b82f387-504c-4dd2-9bfb-fa74a198026e.6509623801|
+---+---------------------------------------------------+
only showing top 5 rows



# Создание витрины "D"

Тоже самое, что и в витрине "B", за исключением генерации идентификатора. Так же различия витрины "D" от "B" будет в количестве строк, при уcловии запущеного цикла в файле "Generation_files.ipynb"

In [14]:
data_mart_D = read_file_JSON().select('INN')
data_mart_D = data_mart_D.select((F.monotonically_increasing_id() + 1).alias('ID'), 'INN')
data_mart_D.show(5, truncate=False)

+-----------+------------+
|ID         |INN         |
+-----------+------------+
|17179869185|388787686306|
|17179869186|275323062033|
|17179869187|694374478861|
|17179869188|976816510709|
|17179869189|884113988041|
+-----------+------------+
only showing top 5 rows



# Создание витрины "E"

Создание функций "search_values_need_dict" и "transformation_values_in_md5" для нахождения значения словаря по ключу и преобразования данных в md5 соответсвенно.

In [ ]:
data_mart_E = read_file_JSON().select(
    (F.monotonically_increasing_id() + 1).alias('ID'),
    transformation_values_in_md5(
        F.expr(search_values_from_cookie('user_phone'))[0]['value'])
    .alias('hash_phone_md5'))

data_mart_E.show(5, truncate=False)

+---+--------------------------------+
|ID |hash_phone_md5                  |
+---+--------------------------------+
|1  |00a3fc817fc3d321da30f3e72407d1f3|
|2  |63a5e9cc46d3cb98a84d7ccd68ba8a74|
|3  |8517c5893c4c2a755ce4c9d7bf9d6554|
|4  |ba94c4fd70a2390c0c85d1250697070f|
|5  |31584837408750ab483819f694c2e0f7|
+---+--------------------------------+
only showing top 5 rows



# Создание витрины "F"

Задание схоже с витриной "F", только необходимой найти значение словоря по другому ключу. Обработка массива словарей, осуществлялась через функцию "F.explode".

In [ ]:
data_mart_F = read_file_JSON().select(
    (F.monotonically_increasing_id() + 1).alias('ID'),
    transformation_values_in_md5(
        F.expr(search_values_from_cookie('user_mail'))[0]['value'])
    .alias('hash_email_md5'))

data_mart_F.show(5, truncate=False)

+---+--------------------------------+
|ID |hash_email_md5                  |
+---+--------------------------------+
|1  |c905c880612181f198da8960ff74dd06|
|2  |e050ba89dce68c3d73a10f68959b6c32|
|3  |c24ad52d2d565a6378cd6daf81fe2628|
|4  |4063185a3633b2798d97ba9c7fa957fe|
|5  |df7e150af4582736610e1d5867fff640|
+---+--------------------------------+
only showing top 5 rows



# Создание витрины "G"

Замена значения через функцию "match_code".

In [ ]:
# pfdzprf yf Mobail
data_mart_G = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)

data_mart_G = data_mart_G.select(
    (F.monotonically_increasing_id() + 1).alias('ID_G'),
    F.expr(search_values_from_cookie('user_uid'))[0]['value'].alias('user_uid'),
    match_code('user_os').alias('match_code'))

data_mart_G.show(5, truncate=False)

+----+--------+----------+
|ID_G|user_uid|match_code|
+----+--------+----------+
|1   |5202886 |null      |
|2   |7831319 |null      |
|3   |1269962 |null      |
|4   |7784937 |null      |
|5   |8905735 |null      |
+----+--------+----------+
only showing top 5 rows



# Создание обобщенной витрины объединяющая предыдущие витрины

Сначала объединяются витрина B и первоначальные данные, чтобы в дальнейшем связать другие витрины с массивом словарей.
Далее объединяюем полученую таблицу с витриной "D", чтобы дополнить список INN для этого использую "FULL JOIN".  Далее объединяем все витрины с главной с помощью "LEFT JOIN".("LEFT JOIN" используется в случает если произошла какая-то коализия данных, чтобы не произошла ситуация с пустым значением "INN")

In [16]:
A = data_mart_A.alias('A')
D = data_mart_D.alias('D')

In [33]:
D.select('INN', F.sha2('INN', 256).alias('sha')).where(F.col('sha') == '77fd79b4a08f57c6c0ac32481297c9501cf1b66a5b5add2b56ca476f253a2f02').show(5, truncate=False)

+------------+----------------------------------------------------------------+
|INN         |sha                                                             |
+------------+----------------------------------------------------------------+
|511432796530|77fd79b4a08f57c6c0ac32481297c9501cf1b66a5b5add2b56ca476f253a2f02|
+------------+----------------------------------------------------------------+



In [35]:
A.join(D, on=(F.sha2(D.INN, 256) == A.data_value)).show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------------+----------------------------------------------------------------+-----------------+------------+-------+--------------+------------------+-------------+----------+---------------+------------+------------+
|raw_cookie                                                                                                                                                                                                                                                                                                                                          

In [ ]:
data_mart_union = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)
#Объединяем с ДФ "В", так как требуется понять к какому INN какие куки принадлежат
data_mart_union = data_mart_union.join(data_mart_B, data_mart_union.INN == data_mart_B.INN, 'full').select(data_mart_B.INN,
                                                                                         data_mart_union.raw_cookie,
                                                                                         data_mart_B.ID_B,
                                                                                         data_mart_union.data_value)
#Объединяем с ДФ "D". из-за того что данных сна считывание сгенирировалось боле в данном ДФ данный будет больше, а те данные которые не объединились с ДВ "Б" будут Null
data_mart_union = data_mart_union.join(data_mart_D, data_mart_union.INN == data_mart_D.INN, 'full').select(data_mart_D.INN,
                                                                                         data_mart_union.raw_cookie,
                                                                                         data_mart_union.data_value,
                                                                                         data_mart_union.ID_B,
                                                                                         data_mart_D.ID_D
                                                                                                          )
#Создание объединённого ДФ
data_mart_union = data_mart_union.join(data_mart_E, transformation_values_in_md5(search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_phone'))) == data_mart_E.hash_phone_md5, 'left')\
               .join(data_mart_F, transformation_values_in_md5(search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_mail'))) == data_mart_F.hash_email_md5, 'left')\
               .join(data_mart_G, search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_uid')) == data_mart_G.user_uid, 'left')\
               .join(data_mart_C, search_dict(data_mart_union.raw_cookie, F.lit('_sa_cookie_a')) == data_mart_C._sa_cookie_a, 'left')\
                                 .select(data_mart_G.user_uid,
                                         data_mart_union.INN,
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_phone')).alias('user_phone'),
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_mail')).alias('user_email'),
                                         data_mart_union.data_value.alias('inn_hash'),
                                         data_mart_E.hash_phone_md5,
                                         data_mart_F.hash_email_md5,
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('org_uid')).alias('org_uid'),
                                         data_mart_union.ID_B,
                                         data_mart_union.ID_D,
                                         data_mart_E.ID_E,
                                         data_mart_F.ID_F,
                                         data_mart_G.ID_G,
                                         data_mart_C.ID_C,
                                         F.array(search_dict(data_mart_union.raw_cookie, F.lit('_sa_cookie_a')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_fa_cookie_a')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_ym_cookie_c')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_fbp'))).alias('array_coockie'))
data_mart_union.show(5, truncate=False)

+--------+------------+---------------+------------------+----------------------------------------------------------------+--------------------------------+--------------------------------+-------+----+-------------+----------+----------+----------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_uid|INN         |user_phone     |user_email        |inn_hash                                                        |hash_phone_md5                  |hash_email_md5                  |org_uid|ID_B|ID_D         |ID_E      |ID_F      |ID_G      |ID_C      |array_coockie                                                                                                                                                                                                                          |
+--------+----